In [41]:
import pandas as pd  # 📊 Manejo de datos en tablas (CSV, Excel) con DataFrames
from selenium import webdriver  # 🌐 Control automático del navegador
from selenium.webdriver.chrome.service import Service  # 🛠️ Configuración del servicio de ChromeDriver
from selenium.webdriver.chrome.options import Options  # ⚙️ Opciones de inicio para Chrome (headless, descargas, etc.)
from webdriver_manager.chrome import ChromeDriverManager  # 🤖 Descarga y gestiona automáticamente el ChromeDriver
from selenium.webdriver.common.by import By  # 🔎 Localización de elementos (ID, NAME, XPATH, etc.)
from selenium.webdriver.support import expected_conditions as EC  # ⏳ Condiciones de espera (elemento visible, clickeable, etc.)
from selenium.webdriver.support.ui import WebDriverWait, Select  # 🕐 WebDriverWait: esperas explícitas | 📋 Select: menús desplegables
from selenium.common.exceptions import NoSuchWindowException  # 🚨 Excepción si la ventana/pestaña ya no existe
import os  # 📂 Manejo de directorios y rutas de archivos
import time  # ⏱️ Pausas y control de tiempo
import re  # 🔤 Expresiones regulares para patrones de texto
import shutil  # 📦 Operaciones con archivos/carpetas (mover, copiar, eliminar)
from datetime import datetime, date, timedelta  # 📅 Manejo de fechas y horas
import datetime
import requests  # 🌐 Peticiones HTTP (descargar archivos, enviar solicitudes con cookies)
import urllib3  # 🌐🔒 Manejo de conexiones HTTP/SSL a bajo nivel (ej. desactivar warnings SSL)
from openpyxl import load_workbook  # 📑 Lectura y escritura avanzada de archivos Excel (.xlsx)
import xlwings as xw  # 🪟 Integración con Excel en tiempo real (controlar hojas, macros, etc.)
import json  # 📄 Manejo de datos en formato JSON (leer/escribir configuraciones o respuestas web)
import subprocess  # 💻 Ejecutar procesos externos o comandos del sistema desde Python
import sys  # ⚙️ Acceso a funciones y variables del intérprete (argumentos, salida, rutas)
import mimetypes  # 📂 Identificación de tipos de archivo según extensión o contenido
import boto3  # ☁️ Cliente de AWS (ej. subir/descargar archivos de S3, usar servicios en la nube)
from botocore.exceptions import ClientError  # 🚨 Manejo de errores al interactuar con servicios AWS
import glob  # 🔍 Búsqueda de archivos por patrones (ej. *.xlsx, *.csv)
import pyautogui  # 🖱️ Automatización de acciones de teclado y mouse (simular clics, escritura, screenshots)

In [42]:
import os, glob, csv

# 1️⃣ Buscar el archivo en Descargas
carpeta = r"C:\Users\jcanalesa\Downloads"
patron  = os.path.join(carpeta, "DGTH-DS-Panetones_Solidarios-_(E-commerce)*.csv")
coincidencias = glob.glob(patron)
if not coincidencias:
    raise FileNotFoundError("No se encontró el CSV con el patrón indicado en Descargas.")
csv_path = coincidencias[0]

# 2️⃣ Encabezados EXACTOS (solo 'Condicion' sin tilde)
target_headers = [
    "Key","Estado del pedido","Fecha del pedido","Nota del cliente","Nombre facturacion",
    "Apellidos facturacion","Empresa facturacion","Direccion lineas 1 y 2 facturacion","Ciudad facturacion",
    "Correo electronico facturacion","Nombre envio","Apellidos envio","Direccion lineas 1 y 2 envio",
    "Ciudad envio","Codigo de provincia envio","Codigo postal envio","Codigo del pais envio",
    "Titulo del metodo de pago","Importe de descuento del carrito","Importe de subtotal del pedido",
    "DIRECCION DE ENVIO","Importe de envio del pedido","Importe reembolsado del pedido","Importe total del pedido",
    "Importe total de impuestos del pedido","DISTRITO","SKU","Articulo","Nombre del articulo","Cantidad",
    "PRECIO UNITARIO","Informacion paneton","Vinculo con PUCP","nro_documento","Telefono facturacion","Tipo",
    "Cantidad total de panetones","Condicion"
]

# 3️⃣ Leer CSV y detectar delimitador
with open(csv_path, "r", encoding="utf-8-sig", newline="") as f:
    sample = f.read(4096)
    f.seek(0)
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",",";","\t","|"])
    except csv.Error:
        class _Dialect(csv.excel): delimiter = ","
        dialect = _Dialect()
    rows = list(csv.reader(f, dialect))

if not rows:
    raise ValueError("El archivo está vacío.")

num_cols = len(rows[0])
if num_cols != len(target_headers):
    raise ValueError(
        f"Número de columnas distinto al de los encabezados dados. "
        f"CSV tiene {num_cols} columnas y se proporcionaron {len(target_headers)} encabezados."
    )

# 4️⃣ Reemplazar la primera fila por los nuevos encabezados
rows[0] = target_headers

# 5️⃣ Sobrescribir el archivo (sin backup)
with open(csv_path, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(
        f,
        delimiter=dialect.delimiter,
        quotechar='"',
        quoting=csv.QUOTE_MINIMAL,
        escapechar="\\"
    )
    writer.writerows(rows)

print("✅ Encabezados reemplazados correctamente (último: 'Condicion').")


✅ Encabezados reemplazados correctamente (último: 'Condicion').


In [43]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------- Configuración ----------
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Si ya iniciaste sesión antes en Google en tu Chrome local (recomendado para evitar CAPTCHA):
# options.add_argument(r"--user-data-dir=C:\Users\TU_USUARIO\AppData\Local\Google\Chrome\User Data")
# options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# ---------- Datos ----------
url = 'https://pucp.kissflow.com/view/dataset/Base_de_datos_panetones_2'
df = pd.read_csv("Kissflow.csv", dtype=str)
usuario = df.loc[0, "usuario"]
contrasena = df.loc[0, "contrasena"]

try:
    # 1️⃣ Abrir la página de Kissflow
    driver.get(url)

    # 2️⃣ Hacer clic en "Continuar con Google"
    google_btn = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'socialLoginItem')]//span[normalize-space()='Google']"))
    )
    google_btn.click()

    # 3️⃣ Esperar a que cargue la página de login de Google y escribir el correo
    email_input = wait.until(EC.element_to_be_clickable((By.ID, "identifierId")))
    email_input.click()
    email_input.clear()
    email_input.send_keys(usuario)

    siguiente_btn = wait.until(EC.element_to_be_clickable((By.ID, "identifierNext")))
    siguiente_btn.click()

    # 4️⃣ Detectar posible CAPTCHA
    time.sleep(2)
    recaptcha_iframes = driver.find_elements(By.XPATH, "//iframe[contains(@src,'recaptcha') or contains(@src,'/challenge')]")
    challenge_elements = driver.find_elements(By.XPATH, "//*[contains(text(),'Escribe el texto') or contains(@aria-label,'captcha')]")

    if recaptcha_iframes or challenge_elements:
        print("⚠️ Se detectó un CAPTCHA. Resuélvelo manualmente y presiona ENTER para continuar.")
        input("Presiona ENTER cuando termines el CAPTCHA...")
    else:
        # 5️⃣ Si no hay CAPTCHA, esperar la contraseña y continuar
        password_input = wait.until(EC.visibility_of_element_located((By.NAME, "password")))
        password_input.click()
        password_input.clear()
        password_input.send_keys(contrasena)

        password_next = wait.until(EC.element_to_be_clickable((By.ID, "passwordNext")))
        password_next.click()

    print("✅ Login a Kissflow completado (verifica visualmente que haya ingresado).")

except Exception as e:
    print("❌ Error durante el flujo:", e)

finally:
    # No cerramos para inspeccionar el navegador
    # driver.quit()
    pass


⚠️ Se detectó un CAPTCHA. Resuélvelo manualmente y presiona ENTER para continuar.
✅ Login a Kissflow completado (verifica visualmente que haya ingresado).


In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os, glob

time.sleep(5)

# --- ya con sesión iniciada ---
# 1️⃣ Click en “Importar CSV”
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Importar CSV']]").click()
time.sleep(1)

# 2️⃣ Click en “Cargar”
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Cargar']]").click()
time.sleep(1)

# 3️⃣ Buscar el input tipo “file” dentro de la opción “Su dispositivo”
input_file = driver.find_element(By.XPATH, "//li[contains(@class,'fileupload')]//input[@type='file']")

# 4️⃣ Buscar el archivo automáticamente en Descargas
carpeta = r"C:\Users\jcanalesa\Downloads"
patron = os.path.join(carpeta, "DGTH-DS-Panetones_Solidarios-_(E-commerce)*.csv")
archivos = glob.glob(patron)

if archivos:
    csv_path = archivos[0]
    input_file.send_keys(csv_path)  # Subida directa, sin ventana
    print("✅ Archivo cargado en Kissflow:", csv_path)
else:
    print("❌ No se encontró el archivo CSV.")


✅ Archivo cargado en Kissflow: C:\Users\jcanalesa\Downloads\DGTH-DS-Panetones_Solidarios-_(E-commerce)102725_16_10_1 (1).csv


In [45]:
time.sleep(5)
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Siguiente']]").click()



In [46]:
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Siguiente']]").click()


In [47]:
# Pausa corta para asegurar carga (ajusta si es necesario)
time.sleep(2)

# 1️⃣ Seleccionar el radio button "whenImporting1"
driver.find_element(By.ID, "whenImporting1").click()
time.sleep(0.5)

# 2️⃣ Seleccionar el radio button "deleteDatasetRows1"
driver.find_element(By.ID, "deleteDatasetRows1").click()
time.sleep(0.5)

# 3️⃣ Seleccionar el radio button "inCaseOfErrors1"
driver.find_element(By.ID, "inCaseOfErrors1").click()
time.sleep(0.5)